In [ ]:
from pandas import Series 
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


from sklearn.metrics import confusion_matrix, auc, roc_auc_score, roc_curve
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import statsmodels.api as sm

import warnings
warnings.filterwarnings("ignore") 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [ ]:
# Зададим цвета
colors = ['#001c57', '#50248f', '#a6a6a6', '#38d1ff','#cc3181']
sns.palplot(sns.color_palette(colors))

# Загрузка данных

In [ ]:
train = pd.read_csv('/kaggle/input/sfdstscoring/train.csv')
test= pd.read_csv('/kaggle/input/sfdstscoring/test.csv')
sample_submission = pd.read_csv('/kaggle/input/sfdstscoring/sample_submission.csv')

In [ ]:
# Соединим train и test в один датасет

train['sample'] = 1  # train
test['sample'] = 0  # test

data = test.append(train, sort=False).reset_index(
    drop=True)

In [ ]:
# Посмотрим на случайные 3 строк базы
display(data.sample(3))
data.info()

In [ ]:
# Посмотрим на типы данных
dtype_data = data.dtypes.reset_index()
dtype_data.columns = ['Count', 'Column Type']
dtype_data.groupby('Column Type').agg('count').reset_index()

In [ ]:
for i, j in enumerate(data.columns):
    print(j, type(data.loc[1][i]))

In [ ]:
# Статистическая информация о базе
data.describe()

In [ ]:
# unique values count, first 10 unique values, null values count, type
data.agg({'nunique', lambda s: s.unique()[:10]})    .append(pd.Series(data.isnull().sum(), name='null'))    .append(pd.Series(data.dtypes, name='dtype'))    .transpose()

Датасет имеет 19 параметров, не включая sample, общее число наблюдений 110148, есть пропуски в education - 478 и default - 36349. При этом education содержит 5 уникальных значений client_id полностью содержит только уникальные значения, можно сказать что нет дубликатов. В целом датасет содержит числовые, бинарные и категориальные признаки. default - наша целевая переменная.

Подробнее по признакам:
* client_id - идентификатор клиента
* education - уровень образования
* sex - пол заемщика
* age - возраст заемщика
* car - флаг наличия автомобиля
* car_type - флаг автомобиля иномарки
* decline_app_cnt - количество отказанных прошлых заявок
* good_work - флаг наличия “хорошей” работы
* bki_request_cnt - количество запросов в БКИ
* home_address - категоризатор домашнего адреса
* work_address - категоризатор рабочего адреса
* income - доход заемщика
* foreign_passport - наличие загранпаспорта
* sna - связь заемщика с клиентами банка
* first_time - давность наличия информации о заемщике
* score_bki - скоринговый балл по данным из БКИ
* region_rating - рейтинг региона
* app_date - дата подачи заявки
* default - флаг дефолта по кредиту

In [ ]:
# Посмотрим на первые 10 строк sample_submission
sample_submission.head(10)

# Предобработка данных

In [ ]:
# Количество пропусков в обучающей базе
train.isna().sum()

In [ ]:
# Количество пропусков в тестовой базе
test.isna().sum()

In [ ]:
# Гистограмма значений признака education, содержащего пропуски, в обучающей базе
train.education.value_counts().plot.barh()

In [ ]:
# Гистограмма значений признака education, содержащего пропуски, в тестовой базе
test.education.value_counts().plot.barh()

In [ ]:
# Избавление от пропусков
# заполним пропуски на наиболее часто встречающееся значение SCH
train.education.fillna('SCH', inplace = True)
test.education.fillna('SCH', inplace = True)

In [ ]:
# Посмотрим на целевую переменную
sns.countplot(train['default'])

Распределение заёмщиков явно неравномерное, недефолтных клиентов заметно больше.

In [ ]:
# попробуем oversampling для устранения дисбаланса
train_0 = train.query('default == 0')
train_1 = train.query('default == 1')
koeff = int(len(train_0)/len(train_1))
for i in range(koeff):
    train = train.append(train_1).reset_index(drop=True)  # объединяем

In [ ]:
# Посмотрим на целевую переменную после
sns.countplot(train['default'])

In [ ]:
# Выведем наименования всех признаков обучающей базы
train.columns

Сгруппируем признаки для упрощения обработки.

In [ ]:
target = 'default'
# числовые переменные
num_cols = ['age', 'decline_app_cnt', 'bki_request_cnt', 'income', 'score_bki', 'region_rating']

# категориальные переменные
cat_cols = ['education', 'work_address', 'home_address', 'sna', 'first_time']

# бинарные переменные
bin_cols = ['sex', 'car', 'car_type', 'good_work', 'foreign_passport']

# Числовые признаки

In [ ]:
# Посмотрим на распределение числовых признаков.
def get_num_info(col, title=None):
    '''Function is called to plot feture distribution'''

    title = title if title is not None else f"Distribution for '{col}"
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5),)
    fig = sm.qqplot(col, fit=True, line='45', ax=ax1)
    fig.suptitle(title, fontsize=20)

    sns.distplot(col.values, bins=20, color=colors[1], ax=ax2)
    sns.violinplot(col.values, color=colors[3], bw=.3, cut=1, linewidth=4)

    ax1.set_title('QQ-plot')
    ax2.set_title('Distribution')
    ax3.set_title('Violinplot')

    plt.show()

In [ ]:
def detect_outliers(data):
    '''Function is called to detect outliers'''
    q1, q3 = np.percentile(sorted(data), [25, 75])

    IQR = q3 - q1

    l_b = q1 - (1.5 * IQR) # lower bound
    u_b = q3 + (1.5 * IQR) # upper bound
    outl_count = len(data[data < l_b]) + len(data[data > u_b])

    print(
        f'Lower Bound: {round(l_b,3)}, Upper Bound {round(u_b,3)}, Outliers Count: {outl_count}')

In [ ]:
for col in num_cols:
    get_num_info(train[col], title=col)
    detect_outliers(train[col])

После построения графиков стало очевидно, что только score_bki имеет распределение близкое к нормальному остальные числовых переменные имеют тяжёлый правый хвост, кроме region_rating.

In [ ]:
# Посмотрим на тепловую карту числовых признаков
sns.heatmap(train[num_cols].corr().abs(), vmin=0, vmax=1, annot = True)

Наибольшая положительная корреляция наблюдается у дохода, затем чуть меньше у количества отказов, что объясняется правилами выдачи кредитов. Но в целом численные признаки скоррелированы слабо.

In [ ]:
# Проверим теперь значимость числовых признаков
imp_num = pd.Series(f_classif(train[num_cols], train['default'])[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

Здесь заметим, что большее значение имеют скоринговый балл по данным из БКИ и количество отказов, в меньшей степени возраст и что удивительно доход также не является наиболее важным значением.

# Бинарные и категориальные признаки

In [ ]:
class Preprocessing:
    def __init__(self, data):
        self.data = data

    def label_encoder(self, column):
        le = LabelEncoder()
        self.data[column] = le.fit_transform(self.data[column])

    def hot_enc(self, column):
        ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
        aux_df = pd.DataFrame(ohe.fit_transform(self.data[[column]]))
        aux_df.columns = ohe.get_feature_names([f'hot_{column}'])
        self.data = self.data.drop(col, axis=1)
        self.data = pd.concat([self.data, aux_df], axis=1)
        return self.data 
    

In [ ]:
# создадим пример класса для тренингового сета
encoder = Preprocessing(train)

In [ ]:
for col in bin_cols:
    encoder.label_encoder(col)

In [ ]:
imp_bol = pd.Series(mutual_info_classif(train[bin_cols], train['default'],
                                        discrete_features=True), index=[bin_cols])
imp_bol.sort_values(inplace=True)
imp_bol.plot(kind='barh')

Самый важные параметры для целевой переменной - наличие загран паспорта и тип машины

In [ ]:
# создадим пример класса для тестового сета 
encoder = Preprocessing(test)

In [ ]:
for col in bin_cols:
    encoder.label_encoder(col)

In [ ]:
# убедимся в преобразовании
display(train.head())
display(test.head())

In [ ]:
def get_boxplot(data, col1, col2, hue=None):
    '''Function is called to plot boxplots'''
    fig, ax = plt.subplots(figsize=(7, 5))
    sns.boxplot(x=col1, y=col2, hue=hue, data=data, palette=colors)
    plt.xticks(rotation=45)
    ax.set_title(f'Boxplot for {col1} and {col2}', fontsize=14)
    plt.show()

In [ ]:
get_boxplot(train, 'education', 'sna', hue='default')

Здесь наблюдается связь между образованием заемщиков и клиентами банка. Как мы видим менее образованные заемщики имеют большую связь с другими клиентами. Может больше кредитов берут из одного коллектива, типа завод и т.п.

In [ ]:
# Заменим значения признака education числами в обеих базах: на 1 - школьное и 0 - высшее
education_dict = {'ACD': 0, 'PGR': 0, 'UGR': 0, 'GRD': 0, 'SCH': 1}
train.education = train['education'].map(education_dict)

train.education.value_counts()

In [ ]:
# Заменим значения признака education числами в тестовой базе
test.education = test['education'].map(education_dict)

test.education.value_counts()

In [ ]:
# Используем encoder для категориальных признаков для лучшей интерпретации.
for col in cat_cols:
    encoder.label_encoder(col)

In [ ]:
imp_cat = pd.Series(mutual_info_classif(train[cat_cols], train['default'],
                                        discrete_features=True), index=[cat_cols])
imp_cat.sort_values(inplace=True)
imp_cat.plot(kind='barh')

In [ ]:
# Посмотрим на тепловую карту категириальных признаков
sns.heatmap(train[cat_cols].corr().abs(), vmin=0, vmax=1, annot = True)

Наибольшая корреляция между дом и раб адресами, чуть меньше между sna & first_time. Но все равно не больше 0.8 поэтому ничего делать не будем.

# Дата

In [ ]:
# Взглянем поближе на признак app_date
train.app_date.head(5), test.app_date.head(5)

In [ ]:
# Сконветируем в более удобный формат даты
train.app_date = pd.to_datetime(train.app_date)
test.app_date = pd.to_datetime(test.app_date)

In [ ]:
# Посмотрим количество уникальных годов в признаке app_date обучающей базы
train.app_date.apply(lambda x: x.year).value_counts()

In [ ]:
# Посмотрим количество уникальных годов в признаке app_date тестовой базы
test.app_date.apply(lambda x: x.year).value_counts()

In [ ]:
# Посмотрим количество уникальных месяцев в признаке app_date обучающей базы
train.app_date.apply(lambda x: x.month).value_counts()

In [ ]:
# Посмотрим количество уникальных месяцев в признаке app_date тестовой базы
test.app_date.apply(lambda x: x.month).value_counts()

# Добавление новых признаков (Feature engineering)

In [ ]:
# Новый признак месяца подачи заявления на кредит
train['month'] = train.app_date.apply(lambda x: x.month)
test['month'] = train.app_date.apply(lambda x: x.month)

cat_cols.append('month')

In [ ]:
# Новый признак количество дней между датой подачи заявления на кредит и датой первой подачи в базе
train['days'] = (train.app_date - train.app_date.min()).dt.days
test['days'] = (test.app_date - test.app_date.min()).dt.days

num_cols.append('days')

In [ ]:
# Прологарифмируем переменные, распределение которых смещено
num_cols_log = ['age', 'bki_request_cnt', 'income', 'decline_app_cnt']

for i in num_cols_log:
    train[i] = np.log(train[i] + 1)

In [ ]:
for i in num_cols_log:
    test[i] = np.log(test[i] + 1)

In [ ]:
# Добавим новые признаки, на основе существующих, которые должны будут улучшить результаты модели
train['bki_age_reg'] = (train['score_bki']/train['age'])*train['region_rating']
test['bki_age_reg'] = (test['score_bki']/test['age'])*test['region_rating']

train['mult_sna_ftime'] = train['sna'] * train['first_time']
test['mult_sna_ftime'] = test['sna'] * test['first_time']

train['edu_and_income'] = (train['education'] + 1) * train['income']
test['edu_and_income'] = (test['education'] + 1) * test['income']

train['success_client'] = (train['foreign_passport'] + 1) * (train['good_work'] + 1) * (train['car'] + 1)
test['success_client'] = (test['foreign_passport'] + 1) * (test['good_work'] + 1) * (test['car'] + 1)

train['very_success_client'] = train['foreign_passport'] * train['good_work'] * train['car']
test['very_success_client'] = test['foreign_passport'] * test['good_work'] * test['car']

train['fpassp_and_gwork'] = train['foreign_passport'] * train['good_work'] 
test['fpassp_and_gwork'] = test['foreign_passport'] * test['good_work']

train['fpassp_and_car'] = train['foreign_passport'] * train['car']
test['fpassp_and_car'] = test['foreign_passport'] * test['car']

train['gwork_and_car'] = train['good_work'] * train['car']
test['gwork_and_car'] = test['good_work'] * test['car']

In [ ]:
# Добавляем новые признаки в соответствующие списки
num_cols.append('bki_age_reg')
num_cols.append('mult_sna_ftime')
num_cols.append('edu_and_income')
cat_cols.append('success_client')
bin_cols.append('very_success_client')
bin_cols.append('fpassp_and_gwork')
bin_cols.append('fpassp_and_car')
bin_cols.append('gwork_and_car')

In [ ]:
# посмотрим на информацию по базам после изменений  
train.info()
test.info()

In [ ]:
# Посмотрим на распределение значений decline_app_cnt в обучающей выборке
train.decline_app_cnt.value_counts()

In [ ]:
# изменим значения признака decline_app_cnt, которые встречаются наиболее редко, единственным значением
train['decline_app_cnt'] = train['decline_app_cnt'].apply(lambda x: x if x < 4 else 4)
test['decline_app_cnt'] = test['decline_app_cnt'].apply(lambda x: x if x < 4 else 4)

In [ ]:
# Изменяем принадлежность к списку признаку decline_app_cnt
num_cols.remove('decline_app_cnt')
cat_cols.append('decline_app_cnt')

# Визуализация и значимость признаков

In [ ]:
# Посмотрим на гистограммы распределения бинарных признаков
for column in bin_cols:
    plt.figure()
    sns.countplot(train[column])
    plt.title(column)
    plt.show()

Бинарные признаки распределены не равномерно, но мы выровняли распределение только по целевой переменной, тут мы ничего делать не будем.

In [ ]:
# boxplots числовых признаков
for column in num_cols:
    plt.figure()
    sns.boxplot(x=train['default'], y=train[column])
    plt.title(column)
    plt.show()

Есть выбросы. Попробуем некоторые исправить.

In [ ]:
# Функция определяет межквартильный интервал и возвращает 1.5 межквартильных расстояния с обеих
# сторон от этого интервала. С её помощью избавимся от выбросов.

def outliers_iqr(ys):
    quartile_1, quartile_3 = np.percentile(ys, [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    return lower_bound, upper_bound

In [ ]:
# Значимость бинарных и категориальных переменных
imp_cat = Series(mutual_info_classif(train[bin_cols + cat_cols], train['default'],
                                     discrete_features =True), index = bin_cols + cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

In [ ]:
# еще раз посмотрим на значимость числовых переменных
imp_num = Series(f_classif(train[num_cols], train['default'])[0],
                 index=num_cols)
imp_num.sort_values(inplace=True)
imp_num.plot(kind='barh')
plt.xlabel('F-value')

In [ ]:
def corr_matrix(data, det=True, pltx=10, plty=10):
    '''Funcion is called for making correlation matrix'''
    
    X = data.corr()
    if det:
        
        evals,evec = np.linalg.eig(X)
        ev_product = np.prod(evals)
    
        print(f'Rank of Matrix: {np.linalg.matrix_rank(X)}')
        print(f'Determinant of matrix: {np.round(ev_product,4)}')
        print(f'Shape of matrix: {np.shape(X)}')
    
    plt.figure(figsize=(pltx,plty))
    sns.heatmap(X,vmin=0,vmax=.9,annot=True,square=True)
    plt.show()

In [ ]:
# посмотрим на корреляцию признаков обучающей базы
corr_matrix(train.drop(['sample'], axis=1), det=False, pltx=20, plty=20)

In [ ]:
# Признаки с высокой корреляцией удалим (выше 0.8 по модулю)
cat_cols.remove('month')

# Подготовка модели

In [ ]:
X_cat = OneHotEncoder(sparse = False).fit_transform(train[cat_cols].values)
X_cat

In [ ]:
Y_cat = OneHotEncoder(sparse = False).fit_transform(test[cat_cols].values)
Y_cat

In [ ]:
# Стандартизация числовых непрерывных переменных на обучающей базе

X_num = StandardScaler().fit_transform(train[num_cols].values)
X_num

In [ ]:
# Стандартизация числовых непрерывных переменных на тестовой базе

Y_num = StandardScaler().fit_transform(test[num_cols].values)
Y_num

In [ ]:
# Объединяем

X = np.hstack([X_num, train[bin_cols].values, X_cat])
Y = train['default'].values

id_test = test['client_id']
test = np.hstack([Y_num, test[bin_cols].values, Y_cat])

In [ ]:
# Разделяем обучающую выборку на тренировочную и валидационную
X_train, X_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.2, random_state=42, shuffle = True)

In [ ]:
# Подбор лучших гиперпараметров для модели

from sklearn.model_selection import GridSearchCV

# Добавим типы регуляризации
penalty = ['l1', 'l2']

# Зададим ограничения для параметра регуляризации
C = np.logspace(0, 4, 10)

# Создадим гиперпараметры
hyperparameters = dict(C=C, penalty=penalty)

model = LogisticRegression()
model.fit(X_train, y_train)

# Создаем сетку поиска с использованием 5-кратной перекрестной проверки
clf = GridSearchCV(model, hyperparameters, cv=5, verbose=0)

best_model = clf.fit(X_train, y_train)

print('Лучшее Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Лучшее C:', best_model.best_estimator_.get_params()['C'])

In [ ]:
# Описываем и обучаем модель
model = LogisticRegression( 
                           C=2.7825594022071245, 
                           class_weight='balanced', 
                           dual=False, 
                           fit_intercept=True, 
                           intercept_scaling=1, 
                           l1_ratio=None, 
                           multi_class='auto', 
                           n_jobs=None, 
                           penalty='l2', 
                           solver='liblinear', 
                           verbose=0, 
                           max_iter=1000)

model.fit(X_train, y_train)

In [ ]:
# Предсказываем значения валидационной базы
Y_predict = model.predict(X_valid)
Y_predict_prob = model.predict_proba(X_valid)[:,1]

In [ ]:
# Предсказываем значения тестовой базы
y_pred_test = model.predict(test)
y_pred_prob_test = model.predict_proba(test)[:,1]

# Предварительные результаты

In [ ]:
# Строим ROC-кривую
fpr, tpr, threshold = roc_curve(y_valid, Y_predict_prob)
roc_auc = roc_auc_score(y_valid, Y_predict_prob)

plt.figure()
plt.plot([0, 1], label='Baseline', linestyle='--')
plt.plot(fpr, tpr, label = 'Regression')
plt.title('Logistic Regression ROC AUC = %0.3f' % roc_auc)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc = 'lower right')
plt.show()

In [ ]:
# Функция для визуализации confusion_matrix
def show_confusion_matrix(y_true, y_pred):
    color_text = plt.get_cmap('PuBu')(0.95)
    class_names = ['Default', 'Non-Default']
    cm = confusion_matrix(y_true, y_pred)
    cm[0,0], cm[1,1] = cm[1,1], cm[0,0]
    df = pd.DataFrame(cm, index=class_names, columns=class_names)
    
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.set(xticks=np.arange(cm.shape[1]), yticks=np.arange(cm.shape[0]), title="Confusion Matrix")
    ax.title.set_fontsize(15)
    sns.heatmap(df, square=True, annot=True, fmt="d", linewidths=1, cmap="PuBu")
    plt.setp(ax.get_yticklabels(), rotation=0, ha="right", rotation_mode="anchor", fontsize=12)
    plt.setp(ax.get_xticklabels(), rotation=0, ha="center", rotation_mode="anchor", fontsize=12)
    ax.set_ylabel('Predicted Values', fontsize=14, color = color_text)
    ax.set_xlabel('Real Values', fontsize=14, color = color_text)
    b, t = plt.ylim()
    plt.ylim(b+0.5, t-0.5)
    fig.tight_layout()
    plt.show()

In [ ]:
# Выведем confusion_matrix
show_confusion_matrix(y_valid, Y_predict)

In [ ]:
# Функция для вывода метрик для оценки качества модели
def all_metrics(y_true, y_pred, y_pred_prob):
    dict_metric = {}
    P = np.sum(y_true==1)
    N = np.sum(y_true==0)
    TP = np.sum((y_true==1)&(y_pred==1))
    TN = np.sum((y_true==0)&(y_pred==0))
    FP = np.sum((y_true==0)&(y_pred==1))
    FN = np.sum((y_true==1)&(y_pred==0))
    
    dict_metric['Positive, P'] = [P,'default']
    dict_metric['Negative, N'] = [N,'non-default']
    dict_metric['True Positive, TP'] = [TP,'correctly identified default']
    dict_metric['True Negative, TN'] = [TN,'correctly identified non-default']
    dict_metric['False Positive, FP'] = [FP,'incorrectly identified default']
    dict_metric['False Negative, FN'] = [FN,'incorrectly identified non-default']
    dict_metric['Accuracy'] = [accuracy_score(y_true, y_pred),'Accuracy=(TP+TN)/(P+N)']
    dict_metric['Precision'] = [precision_score(y_true, y_pred),'Precision = TP/(TP+FP)'] 
    dict_metric['Recall'] = [recall_score(y_true, y_pred),'Recall = TP/P']
    dict_metric['F1-score'] = [f1_score(y_true, y_pred),'Harmonical mean of Precision и Recall']
    dict_metric['ROC_AUC'] = [roc_auc_score(y_true, y_pred_prob),'ROC AUC Score']    

    temp_df = pd.DataFrame.from_dict(dict_metric, orient='index', columns=['Value', 'Description'])
    display(temp_df)

In [ ]:
# Выведем метрики качества модели
all_metrics(y_valid, Y_predict, Y_predict_prob)

In [ ]:
# Обучаем модель на всей обучающей базе
main_model = LogisticRegression( 
                           C=2.7825594022071245, 
                           class_weight='balanced', 
                           dual=False, 
                           fit_intercept=True, 
                           intercept_scaling=1, 
                           l1_ratio=None, 
                           multi_class='auto', 
                           n_jobs=None, 
                           penalty='l2', 
                           solver='liblinear', 
                           verbose=0, 
                           max_iter=1000)
main_model.fit(X, Y)

# Предсказываем значения тестовой базы
y_pred_test = main_model.predict(test)
y_pred_prob_test = main_model.predict_proba(test)[:,1]

# Submission

In [ ]:
# Записываем предсказанные моделью вероятности дефолта заемщиков из тестовой базы в отдельный файл
new_sample_submission = pd.DataFrame({'client_id': id_test,
                              'default': y_pred_prob_test})
new_sample_submission.to_csv('submission.csv', index=False)

new_sample_submission.head(10)